In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Irritable_bowel_syndrome_(IBS)"
cohort = "GSE63379"

# Input paths
in_trait_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)"
in_cohort_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)/GSE63379"

# Output paths
out_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE63379.csv"
out_gene_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE63379.csv"
out_clinical_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE63379.csv"
json_path = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide Expression Profiling in Irritable Bowel Syndrome"
!Series_summary	"Differential gene expression profiling in peripheral blood mononuclear cells (PBMCs) was performed using Human Transcriptome Array 2 (HTA2)"
!Series_overall_design	"Expression profiles of peripheral blood mononuclear cell (PBMCs) from 35 IBS samples and 32 healthy control was assessed."
Sample Characteristics Dictionary:
{0: ['disease status: healthy', 'disease status: IBS'], 1: ['tissue: blood'], 2: ['cell type: peripheral blood mononuclear cells']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on series summary and design, this dataset contains gene expression data from PBMCs using HTA2 array
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (IBS status): available in row 0
trait_row = 0
# No age data available
age_row = None
# No gender data available
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert IBS status to binary: 1 for IBS, 0 for healthy control"""
    if value is None:
        return None
    
    # Extract value after colon if exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'ibs':
        return 1
    elif value.lower() == 'healthy':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to float"""
    # Not applicable as age data is not available
    return None

def convert_gender(value):
    """Convert gender to binary: 1 for male, 0 for female"""
    # Not applicable as gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Read clinical data
    clinical_data_file = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_file):
        clinical_data = pd.read_csv(clinical_data_file)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of clinical data:")
        print(preview)
        
        # Save clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 58
Header line: "ID_REF"	"GSM1547708"	"GSM1547709"	"GSM1547710"	"GSM1547711"	"GSM1547712"	"GSM1547713"	"GSM1547714"	"GSM1547715"	"GSM1547716"	"GSM1547717"	"GSM1547718"	"GSM1547719"	"GSM1547720"	"GSM1547721"	"GSM1547722"	"GSM1547723"	"GSM1547724"	"GSM1547725"	"GSM1547726"	"GSM1547727"	"GSM1547728"	"GSM1547729"	"GSM1547730"	"GSM1547731"	"GSM1547732"	"GSM1547733"	"GSM1547734"	"GSM1547735"	"GSM1547736"	"GSM1547737"	"GSM1547738"	"GSM1547739"	"GSM1547740"	"GSM1547741"	"GSM1547742"	"GSM1547743"	"GSM1547744"	"GSM1547745"	"GSM1547746"	"GSM1547747"	"GSM1547748"	"GSM1547749"	"GSM1547750"	"GSM1547751"	"GSM1547752"	"GSM1547753"	"GSM1547754"	"GSM1547755"	"GSM1547756"	"GSM1547757"	"GSM1547758"	"GSM1547759"	"GSM1547760"	"GSM1547761"	"GSM1547762"	"GSM1547763"	"GSM1547764"	"GSM1547765"	"GSM1547766"	"GSM1547767"	"GSM1547768"	"GSM1547769"	"GSM1547770"	"GSM1547771"	"GSM1547772"	"GSM1547773"	"GSM1547774"
First data line: "2824546_st"	11.72	12.16	12.45	11.97	12.73	13.05	12.72	13.11	

Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data, I can see they follow the format 
# like "2824546_st", which suggests these are probe IDs from a microarray platform (likely Affymetrix)
# rather than standard human gene symbols.
#
# These identifiers need to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns containing gene IDs and gene symbols
# After examining the gene annotation data, I can see:
# - The probe IDs in the gene expression data end with "_st" (e.g., "2824546_st")
# - The 'ID' column in annotation contains probe identifiers with a different format (e.g., "TC01000001.hg.1")
# - The 'gene_assignment' column contains gene symbol information

# First, let's look at the format of IDs in the gene expression data vs. annotation data
print("First few probe IDs in gene expression data:")
print(gene_data.index[:5])

# We need to check if the annotation data's ID is comparable to the gene data's ID format
# Let's compare a small sample

# Let's inspect the gene_assignment column which likely has gene symbol information
print("\nGene assignment example:")
print(gene_annotation['gene_assignment'].iloc[0])

# 2. Create a mapping dataframe with probe IDs and corresponding gene symbols
# Since the IDs don't directly match, we need to find a way to map between them
# Let's check if there's a direct way to map using other columns

# The gene expression data probes end with "_st" which suggests they're from Affymetrix platform
# We need to examine the gene annotation more carefully to find the right columns

# Extract probe ID and gene symbol mapping
# In this case, we'll use the 'ID' column as the probe ID and extract gene symbols from 'gene_assignment'
mapping_df = pd.DataFrame({'ID': gene_annotation['ID'].astype(str)})
mapping_df['Gene'] = gene_annotation['gene_assignment']

# 3. Apply gene mapping to convert probe measurements to gene expression values
# This will extract gene symbols from the gene_assignment text and apply the mapping
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the mapped gene expression data
print("\nGene expression data after mapping (first 5 rows, 5 columns):")
print(gene_data.iloc[:5, :5])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


First few probe IDs in gene expression data:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st'], dtype='object', name='ID')

Gene assignment example:
NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102



Gene expression data after mapping (first 5 rows, 5 columns):
         GSM1547708  GSM1547709  GSM1547710  GSM1547711  GSM1547712
Gene                                                               
A-        18.412500   18.445417   18.259167   18.320000   18.404167
A-2        1.498000    1.492000    1.498000    1.494000    1.482000
A-52       3.243333    3.263333    3.356667    3.276667    3.306667
A-575C2    2.292500    2.227500    2.232500    2.227500    2.235000
A-E        2.121667    2.066667    2.055000    2.073333    2.061667


Gene expression data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE63379.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the normalized gene data 
gene_data = pd.read_csv(out_gene_data_file, index_col=0)

# 2. Re-extract clinical features from the SOFT file to get proper clinical data
# Use the actual clinical data from the matrix file properly
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# 3. Create a correct clinical features dataframe
# First inspect what's in the clinical data
clinical_data_dict = get_unique_values_by_row(clinical_data)
print("Clinical data structure:")
print(clinical_data_dict)

# Based on the sample characteristics dictionary shown previously, 
# extract and process clinical features
selected_clinical_df = pd.DataFrame()

# Process disease state row manually to ensure correct mapping
disease_row = clinical_data.iloc[trait_row]
samples = [col for col in disease_row.index if col != "!Sample_geo_accession"]
trait_values = []

for sample in samples:
    value = disease_row[sample]
    if pd.isna(value):
        trait_values.append(None)
    else:
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "IBS" in value:
            trait_values.append(1)  # IBS is our target trait
        elif "IBD" in value:
            trait_values.append(0)  # IBD is the control
        else:
            trait_values.append(None)

# Create dataframe with processed values
selected_clinical_df[trait] = trait_values
selected_clinical_df.index = samples

# Save the corrected clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Corrected clinical data saved to {out_clinical_data_file}")

# 4. Link the clinical and genetic data
linked_data = pd.DataFrame()
# Transpose gene data to have samples as rows and genes as columns
gene_data_t = gene_data.T
# Verify alignment of sample IDs between clinical and gene data
common_samples = list(set(selected_clinical_df.index) & set(gene_data_t.index))
if common_samples:
    gene_data_filtered = gene_data_t.loc[common_samples]
    clinical_data_filtered = selected_clinical_df.loc[common_samples]
    # Join the data
    linked_data = pd.concat([clinical_data_filtered, gene_data_filtered], axis=1)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Alternative linking approach if sample IDs don't directly match
    print("No common sample IDs found. Attempting alternative linking...")
    # The GSM ids in gene data columns may correspond to the sample IDs
    clinical_data_reset = selected_clinical_df.reset_index()
    clinical_data_reset.columns = ["Sample"] + list(clinical_data_reset.columns[1:])
    gene_data_cols = list(gene_data.columns)
    
    # Create merged dataframe
    data_dict = {trait: []}
    # Add trait values
    for col in gene_data_cols:
        sample_idx = clinical_data_reset.index[clinical_data_reset["Sample"] == col] if "Sample" in clinical_data_reset.columns else []
        if len(sample_idx) > 0:
            data_dict[trait].append(clinical_data_reset.loc[sample_idx[0], trait])
        else:
            data_dict[trait].append(None)
    
    # Add gene expression values
    for gene in gene_data.index:
        data_dict[gene] = list(gene_data.loc[gene])
    
    linked_data = pd.DataFrame(data_dict, index=gene_data_cols)
    print(f"Alternative linked data shape: {linked_data.shape}")

# 5. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 6. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 7. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from patients with IBS and IBD, examining effects of relaxation response mind-body intervention."
)

# 8. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Clinical data structure:
{0: ['disease status: healthy', 'disease status: IBS'], 1: ['tissue: blood'], 2: ['cell type: peripheral blood mononuclear cells']}
Corrected clinical data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE63379.csv
Linked data shape: (67, 71529)


Data after handling missing values: (35, 71529)
Quartiles for 'Irritable_bowel_syndrome_(IBS)':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is severely biased.

Data was determined to be unusable and was not saved
